In [1]:
news_data = []
with open("news_summary.txt", "r", encoding="utf-8") as file:
    news_data = file.readlines()
    print(f"loaded {len(news_data)}")

loaded 147


In [2]:
import ollama
embedding_model = "hf.co/CompendiumLabs/bge-base-en-v1.5-gguf"
language_model = "qwen3:latest" # qwen is goated going through 
 
vector_db = []

In [3]:
def add_chunk_to_database(chunk):
    embedding = ollama.embed(model=embedding_model, input=chunk)["embeddings"][0]
    vector_db.append((chunk, embedding))

In [4]:
for i, chunk in enumerate(news_data):
    add_chunk_to_database(chunk)
    print(f"added chunk {i+1}/{len(news_data)} to the database")

added chunk 1/147 to the database
added chunk 2/147 to the database
added chunk 3/147 to the database
added chunk 4/147 to the database
added chunk 5/147 to the database
added chunk 6/147 to the database
added chunk 7/147 to the database
added chunk 8/147 to the database
added chunk 9/147 to the database
added chunk 10/147 to the database
added chunk 11/147 to the database
added chunk 12/147 to the database
added chunk 13/147 to the database
added chunk 14/147 to the database
added chunk 15/147 to the database
added chunk 16/147 to the database
added chunk 17/147 to the database
added chunk 18/147 to the database
added chunk 19/147 to the database
added chunk 20/147 to the database
added chunk 21/147 to the database
added chunk 22/147 to the database
added chunk 23/147 to the database
added chunk 24/147 to the database
added chunk 25/147 to the database
added chunk 26/147 to the database
added chunk 27/147 to the database
added chunk 28/147 to the database
added chunk 29/147 to the dat

In [5]:
# use cosine similiarity to find close chunks 
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [6]:
def retrieve(query, top_n=10):
    query_embedding = ollama.embed(model=embedding_model, input=query)["embeddings"][0]
    similarities = []
    for chunk, embedding in vector_db:
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))
    # sort by similarity in descending order, because higher similarity means more relevant chunks
    similarities.sort(key=lambda x: x[1], reverse=True)
    # finally, return the top N most relevant chunks
    return similarities[:top_n]

In [7]:
input_query = input("Ask me anything about stocks! ")
retrieved_knowledge = retrieve(input_query)

print("retrieved knowledge: ")
for chunk, similarity in retrieved_knowledge:
    print(f" - (similarity: {similarity:.2f}) {chunk}")

context_text = '\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])

instruction_prompt = f"""
You are a professional stock market analysis assistant with access to real-time news via RAG.
You must always ground your commentary in the retrieved articles.
IMPORTANT: Do NOT show your thinking process. Do NOT use <think> tags or explain your reasoning. Just give your final response directly.
Rules:
1. Use ONLY the retrieved news articles as your source of information.
2. Always rely on the LATEST news provided in the feed. Do not invent or fabricate numbers, prices, or events.
3. Respond in ONE well-structured paragraph that flows naturally, as if written for a financial news briefing.
4. No bullet points, no lists, no over-formatting.
5. The response should feel like a concise but insightful market recap.
6. Maintain a professional, analytical tone—clear, objective, and polished.
7. Integrate facts from the news seamlessly into the commentary, highlighting key drivers of market sentiment.
8. If the news is routine, frame it as stability or consolidation; if it is significant, highlight its impact on markets, sectors, or investor sentiment.
9. Do NOT speculate recklessly or use slang; remain focused on credible, news-based insights.
10. ALWAYS end by saying this is not financial advice do not trade off what I say. 
Your goal is to deliver a single, cohesive, professional market update that summarizes the latest news into a clear, authoritative narrative.
{context_text}
"""

retrieved knowledge: 
 - (similarity: 0.61) Microsoft Corp. MSFT has successfully evaded a substantial antitrust penalty as the European Union has agreed to the tech behemoth’s commitment to separate its Teams platform from its productivity applications. EU Clears Microsoft After Teams Separation PledgeThe EU, on Friday, announced that it has agreed to Microsoft’s commitments to separate its Teams workplace communication platform from its popular productivity apps. Microsoft will provide versions of its Office 365 and Microsoft 365 software suites at a discounted rate without including Teams. The EU launched its antitrust investigation into Microsoft in July 2023 after Salesforce-owned Slack, a competing chat service to Teams, filed a complaint. This recent move by Microsoft to unbundle Teams from its productivity apps is another step in its ongoing compliance with EU regulations.

 - (similarity: 0.59) The following companies are leaders in cloud services and data management that coul

In [8]:
stream = ollama.chat(
    model=language_model,
    messages=[
        {'role': 'system', 'content': instruction_prompt},
        {'role': 'user', 'content': input_query},
    ],
    stream=True
)

print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Chatbot response:
<think>
Okay, the user is asking, "tell me whats new with microsoft." Let me check the provided news articles to find the latest updates on Microsoft.

First, there's an article about the EU clearing Microsoft after they separated Teams from their productivity apps. That's a recent development, so that's important. Then, there's mention of Microsoft's Azure cloud revenue growing by 39% YoY, which is faster than competitors. Also, the EU antitrust case resolution is a key point. 

Another article talks about Microsoft's dividend potential and strong financial metrics like ROE and gross profit. The dividend growth potential is a positive point. Also, the article about Microsoft's AI initiatives, like Azure AI Foundry, and partnerships with Nebius for data centers. 

There's also a comparison with Apple, but the user might be more interested in Microsoft's own news. The Coatue hedge fund's prediction about Microsoft being a $5.7 trillion company by 2030 is a significant 